# Custom Training StyleGan2-ADA

This notebook is made to work with Jupyter Notebooks on a dedicated server. If you would like to train StyleGAN2 in Colab please use [Derrick Schultz's original notebook](https://github.com/dvschultz/ai/blob/master/StyleGAN2.ipynb).

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

import tensorflow as tf
print(tf.__version__)

with tf.Session() as sess:
  devices = sess.list_devices()
  for device in devices:
    print(device)

1.14.0
_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3291454608227213190)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14540349452324919188)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7971792487, 5772944377220684582)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 13877897758306955812)


In [2]:
!nvidia-smi

Sun Nov 15 22:54:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   30C    P8     6W / 180W |     11MiB /  8119MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Clone Repo locally

Run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [3]:
import os
if os.path.isdir("stylegan2-ada"):
    %cd "stylegan2-ada"
else:
    #install script
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/home/guma/Documents/App/ML/datacrunch-stylegan2-ada/stylegan2-ada


In [ ]:
!git config user.name "test"
!git config user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

## Download dataset

Remember that the dataset has to be preprocessed and __resize at the correct size (in our case 1024x1024)__. You can use this handy library to do that: https://github.com/dvschultz/dataset-tools

Upload your dataset as a zip file to Google drive and note the file ID.

In [4]:
import gdown

# file_id = 'YOUR-FILE-ID'
file_id = '1_eOre42oqWaMfx8riyAE-p5IFxi3EZRB'

url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1Hv0wiw5K5L4LxDAPqMsESm2jLeav5NYD
To: /home/guma/Documents/App/ML/datacrunch-stylegan2-ada/stylegan2-ada/flowers-sq-1024.zip
2.91GB [10:44, 4.51MB/s]


'flowers-sq-1024.zip'

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to continue training, skip these cells.**

We need to convert our image dataset to a format that StyleGAN2-ADA can read from.

First we unzip our images:

In [5]:
import patoolib

# zip_path = "YOUR-ZIP-FILE.zip"
zip_path = "flowers-sq-1024-small.zip"

!mkdir dataset_imgs
patoolib.extract_archive(zip_path, outdir='dataset_imgs')

patool: Extracting flowers-sq-1024.zip ...
patool: ... flowers-sq-1024.zip extracted to `dataset_imgs'.


'dataset_imgs'

Now that your image dataset is ready, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while. 

In [8]:
#update this to the path to your image folder
dataset_path = "dataset_imgs/flowers-sq-1024-small"
#give your dataset a name
dataset_name = "flowers-sq-1024-small"

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "dataset_imgs/flowers-sq-1024"
Creating dataset "./datasets/flowers-sq-1024"
Added 40 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [9]:
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--metrics LIST]
                [--metricdata PATH]
                [--cfg {auto,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline}]
                [--gamma FLOAT] [--kimg INT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional arguments:
  -h, --help            show this help message and exit

general options:
  --outdir DIR          Where to save the results (required)
  --gpus INT            Number of GPUs to use (default: 1 gpu)
  --sn

In [12]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "flowers-sq-1024-small"
#how often should the model generate samples and a .pkl file
snapshot_count = 4
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics? 
metric_list = None
#augments
augs = "bg"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
# resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"
resume_from = "ffhq1024"

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from}


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 16384,
    "fmap_max": 512,
    "mapping_layers": 8,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 16384,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.002
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.002
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 10
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1
    },
    "tune_kimg": 100
  }

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.